# 机器学习实验报告-lab1-KNN
**姓名：** 郭裕彬
**学号：** 2114052
**专业：** 物联网工程
# 实验要求
 **题目：** 基于KNN 的手写数字识别 实验条件：给定semeion手写数字数据集，给定kNN分类算法
 1. 基本要求：编程实现kNN算法,要求使用留一法,求得在给出不同k值（1,3,5）情况下,kNN算法对⼿写数字的识别精度.
 2. 中级要求：与机器学习包中的kNN分类结果进行对比
 3. 提高要求：采用旋转等手段对原始数据进行处理，扩增数据量，采用CNN或其他深度学习方法实现手写体识别

### 导入使用的包

In [1]:
import numpy as np
from sklearn.model_selection import LeaveOneOut, train_test_split
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras import layers, models
import cv2

### 导入并处理数据集

In [2]:
#导入数据
def ImgtoMat(filename):
    f = open( filename )
    ss = f.readlines()
    l = len(ss)
    f.close()
    returnMat = np.zeros((l,256))
    returnClassVector = np.zeros((l,1))
    for i in range(l):
        s1 = ss[i].split()
        for j in range(256):
            returnMat[i][j] = np.float64(s1[j])
        clCount = 0
        for j in range(256, 266):
            if s1[j] == '1':
                clCount = j-256
                break
        returnClassVector[i] = clCount
    return returnMat, returnClassVector
X,y = ImgtoMat('semeion.data')
np.shape(X), np.shape(y)

((1593, 256), (1593, 1))

### 基本要求
编程实现kNN算法,要求使用留一法,求得在给出不同k值（1,3,5）情况下,kNN算法对手写数字的识别精度.

In [3]:
#knn算法手动实现
def knn(X, Y, neighbors):
    #留一法分割
    loo = LeaveOneOut()
    testRes = []
    corCount = 0
    for trainIndex,testIndex in loo.split(X, Y):
        xTrain, xTest, yTrain, yTest = X[trainIndex], X[testIndex], Y[trainIndex], Y[testIndex]
        trainVol = xTrain.shape[0]
        testVol = xTest.shape[0]
        #差异矩阵
        diffMat = np.tile(xTest[0], (trainVol, 1)) - xTrain
        #获得标准差距离并排序
        sqDiffMat = diffMat ** 2
        sqDistance = sqDiffMat.sum(axis=1)
        distance = sqDistance ** 0.5
        sortDistance = np.argsort(distance)
        labelCount = []
        for j in range(neighbors):  # 考察k近邻属于哪些类
            labelCount.append(yTrain[sortDistance[j]][0])
        classifyRes = Counter(labelCount)  # 把k近邻中最多的那个标签作为分类结果
        classifyRes = classifyRes.most_common(2)[0][0]
        testRes.append(classifyRes)
        if yTest[0] == classifyRes:#分类正确
            corCount += 1
    corRate = corCount / 1593
    print('k={0}时，测试个数为1593  正确个数为：{1}  准确率为：{2}'.format(neighbors,corCount,corRate))
    return corRate

测试结果

In [4]:
for i in [1,3,5]:
    knn(X, y, i)

k=1时，测试个数为1593  正确个数为：1459  准确率为：0.9158819836785939
k=3时，测试个数为1593  正确个数为：1464  准确率为：0.9190207156308852
k=5时，测试个数为1593  正确个数为：1458  准确率为：0.9152542372881356


### 中级要求
与机器学习包中的kNN分类结果进行对比

In [5]:
def sklearnKnn(X, Y, neighbors):
    loo = LeaveOneOut()
    testRes = []
    corCount = 0
    for trainIndex, testIndex in loo.split(X, Y):
        xTrain, xTest, yTrain, yTest = X[trainIndex], X[testIndex], Y[trainIndex], Y[testIndex]
        neigh = KNeighborsClassifier(n_neighbors=neighbors)
        neigh.fit(xTrain, yTrain.ravel())
        predict_y = neigh.predict(xTest)
        if yTest[0] == predict_y:
                corCount += 1
    corRate = corCount / Y.shape[0]
    print('k={0}时，测试个数为1593  正确个数为：{1}  准确率为：{2}'.format(neighbors, corCount, corRate))
    return corRate

测试结果

In [6]:
for i in [1,3,5]:
    sklearnKnn(X, y, i)

k=1时，测试个数为1593  正确个数为：1461  准确率为：0.9171374764595104
k=3时，测试个数为1593  正确个数为：1439  准确率为：0.9033270558694287
k=5时，测试个数为1593  正确个数为：1440  准确率为：0.903954802259887


### 提高要求
采用旋转等手段对原始数据进行处理，扩增数据量，采用CNN或其他深度学习方法实现手写体识别

In [67]:
model = models.Sequential([
    layers.Reshape((16, 16, 1), input_shape=(256,)),
    layers.Conv2D(36, (5, 5), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(36, (5, 5), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
#旋转扩增训练集
def rotate_image(image, angle):
    center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result
#cnn方法
def cnn(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    augmented_X_train = []
    augmented_y_train = []
    for i in range(len(X_train)):
        image = X_train[i].reshape(16, 16)  # 将扁平的图像数据重塑为16x16
        for angle in [-15, 0, 15]:  # 旋转角度范围
            rotated_image = rotate_image(image, angle)
            augmented_X_train.append(rotated_image.flatten())  # 重塑为扁平的数据
            augmented_y_train.append(y_train[i])
    X_train = np.vstack([X_train, np.array(augmented_X_train)])
    y_train = np.concatenate([y_train, np.array(augmented_y_train)])
    #标准化
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print('cnn准确率为{0}'.format(test_acc))

调整参数获得较好的测试结果

In [68]:
cnn (X, y)

Epoch 1/20
160/160 [==============================] - 1s 4ms/step - loss: 1.1831 - accuracy: 0.6015 - val_loss: 0.2529 - val_accuracy: 0.9373
Epoch 2/20
160/160 [==============================] - 1s 4ms/step - loss: 0.4493 - accuracy: 0.8577 - val_loss: 0.1544 - val_accuracy: 0.9592
Epoch 3/20
160/160 [==============================] - 1s 4ms/step - loss: 0.3053 - accuracy: 0.9036 - val_loss: 0.1105 - val_accuracy: 0.9592
Epoch 4/20
160/160 [==============================] - 1s 4ms/step - loss: 0.2313 - accuracy: 0.9219 - val_loss: 0.0821 - val_accuracy: 0.9687
Epoch 5/20
160/160 [==============================] - 1s 4ms/step - loss: 0.1899 - accuracy: 0.9398 - val_loss: 0.0727 - val_accuracy: 0.9655
Epoch 6/20
160/160 [==============================] - 1s 4ms/step - loss: 0.1694 - accuracy: 0.9443 - val_loss: 0.0697 - val_accuracy: 0.9749
Epoch 7/20
160/160 [==============================] - 1s 4ms/step - loss: 0.1311 - accuracy: 0.9564 - val_loss: 0.0613 - val_accuracy: 0.9875
Epoch 